In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 9.4 MB/s eta 0:00:00


In [ ]:
import pulp as lp
import pandas as pd

# Load data
advertiser_data = pd.DataFrame({
    'Month': ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
              '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12'],
    'Advertisers': [1024, 996, 1024, 972, 943, 859, 915, 919, 1063, 1499, 1478, 1369],
    'Avg_Budget': [84301, 86866, 86477, 86291, 86199, 84493, 86704, 84993, 87440, 86020, 85232, 86105]
})

# Parameters
initial_agents = 652
annual_salary = 77721
monthly_salary = annual_salary / 12
firing_cost_perc = 0.4
max_adv_per_agent = 10
expected_uplift = 0.135
n_months = len(advertiser_data)

# Initialize PuLP problem
prob = lp.LpProblem("Dynamic_Staffing_Optimization", lp.LpMinimize)

# Decision Variables
months = list(range(n_months))
A = lp.LpVariable.dicts("Available_Agents", months, lowBound=0, cat='Integer')
N = lp.LpVariable.dicts("New_Hires", months, lowBound=0, cat='Integer')
F = lp.LpVariable.dicts("Fired", months, lowBound=0, cat='Integer')
S = lp.LpVariable.dicts("Assigned_Advertisers", months, lowBound=0, cat='Integer')
W = lp.LpVariable.dicts("Waiting_Pool", months, lowBound=0, cat='Integer')


In [ ]:
# Initial Conditions
prob += A[0] == initial_agents
W_prev = 0

# Add constraint to force assignment of January advertisers
jan_advertisers = advertiser_data['Advertisers'][0]
prob += S[0] == jan_advertisers  # Enforce assignment in January

# Constraints
for m in months:
    if m == 0:
        T = A[m]
    else:
        T = A[m] + N[m-1] - F[m-1]

    prob += S[m] <= T * max_adv_per_agent
    prob += W[m] == W_prev + advertiser_data['Advertisers'][m] - S[m]
    prob += W[m] >= 0

    prob += N[m] <= 0.2 * A[m]
    prob += F[m] <= 0.25 * A[m]

    if m == 0:
        prob += T * 10 >= S[m]
    else:
        prob += T * 10 >= S[m] + S[m-1]

    if m < n_months - 1:
        prob += A[m+1] == A[m] + N[m] - F[m]

    W_prev = W[m]

# Objective
total_cost = lp.lpSum(
    [A[m] * monthly_salary + F[m] * firing_cost_perc * annual_salary
     - S[m] * advertiser_data['Avg_Budget'][m] * expected_uplift
     for m in months]
)
prob += total_cost

# Solve
prob.solve()



1

In [ ]:
# Results
results = []
for m in months:
    results.append({
        'Month': advertiser_data['Month'][m],
        'Available Agents': A[m].varValue,
        'New Hires': N[m].varValue,
        'Fired': F[m].varValue,
        'Assigned Advertisers': S[m].varValue,
        'Total Cost ($)': round(A[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2),
        'Revenue Uplift ($)': round(S[m].varValue * advertiser_data['Avg_Budget'][m] * expected_uplift, 2)
    })

results_df = pd.DataFrame(results)
total_annual_cost = results_df['Total Cost ($)'].sum()
total_uplift = results_df['Revenue Uplift ($)'].sum()
net_profit = total_uplift - total_annual_cost

print(results_df.to_string(index=False))
print(f"\nTotal Annual Cost: ${total_annual_cost:,.2f}")
print(f"Total Revenue Uplift: ${total_uplift:,.2f}")
print(f"Net Profit: ${net_profit:,.2f}")

  Month  Available Agents  New Hires  Fired  Assigned Advertisers  Total Cost ($)  Revenue Uplift ($)
2025-01             652.0        0.0  163.0                1024.0      9290250.20         11653770.24
2025-02             489.0        0.0  122.0                 996.0      6959915.55         11680002.36
2025-03             367.0        0.0   91.0                1024.0      5206011.65         11954580.48
2025-04             276.0        0.0   54.0                 826.0      3466356.60          9622309.41
2025-05             222.0        0.0    0.0                 854.0      1437838.50          9937882.71
2025-06             222.0        0.0    0.0                   7.0      1437838.50            79845.89
2025-07             222.0        0.0    0.0                2002.0      1437838.50         23433490.08
2025-08             222.0        0.0    0.0                 218.0      1437838.50          2501343.99
2025-09             222.0       23.0    0.0                1764.0      1437838.50 